# Проект по поиску токсичных комментариев

Интернет-магазин «Викишоп» запускает новый сервис, чтобы пользователи могли редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину запросил инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Цель проекта: обучить модель классифицировать комментарии на позитивные и негативные. 

Задачи:
1. Загрузить и подготовить данные
2. Обучить разные модели
3. Построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**
Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак. В нашем распоряжении набор данных с разметкой о токсичности правок.


## Подготовка

In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics
import nltk
import pymorphy2
import spacy
import re

from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import torch
import transformers

from tqdm import notebook

from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

# Install spaCy (run in terminal/prompt)
import sys
!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 4.4 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
comments = pd.read_csv('/datasets/toxic_comments.csv')
comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
comments.shape

(159571, 2)

Посмотрим, насколько классы, представленные в целевом признаки сбалансированы между собой.

In [5]:
display(comments['toxic'].value_counts())
class_ratio = comments['toxic'].value_counts()[0] / comments['toxic'].value_counts()[1]
class_ratio

0    143346
1     16225
Name: toxic, dtype: int64

8.834884437596301

Классы несбалансированы. Отношение 1:8.83. 

Поскольку у меня умирало ядро на всем объеме датасета, сделала на части его, которая уже позволяла хоть одной из модели получить показатель F1 больше 0,75 

In [6]:
comments_sample = comments.sample(100000).reset_index(drop=True) 

Подготовим данные для векторизации.

Проведем токенизацию и лемматизацию при помощи nlp

In [7]:
%time
nlp= spacy.load('en_core_web_sm')

def lemmatize(text): 
    text=text.lower()
    nlp_text=nlp(text)
    lemm_text= ' '.join(token.lemma_ for token in nlp_text)
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return cleared_text
comments_sample['lemm']=comments_sample['text'].apply(lemmatize)
comments_sample['lemm'].sample(5)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs


77700             john smith              mar        utc  
6068         I apologize for my inappropriate language ...
83590    your stalker friend be back   the hostile user...
6077     infobox work    hi   hopefully I will have the...
90383    that s a personal attack   my name do not offe...
Name: lemm, dtype: object

добавим стопслова.

In [8]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
len(stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

## Обучение

Разобьем выборки на обучающую и тестовую:

In [9]:
features_train, features_test, target_train, target_test = train_test_split(comments_sample['lemm'], comments_sample['toxic'], 
                                                    test_size=0.2, random_state=12345)

Проведём векторизацию корпусов с помощью TfidfVectorizer, заодно удалим стоп-слова.

In [10]:
tf_idf_vec = TfidfVectorizer(ngram_range=(1,1), stop_words=stopwords, min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [11]:
features_train_vec = tf_idf_vec.fit_transform(features_train)

In [12]:
features_test_vec = tf_idf_vec.transform(features_test)

In [13]:
base_predicts = pd.Series(data=np.zeros((len(target_test))), index=target_test.index, dtype='int16')
base_accuracy = accuracy_score(target_test, base_predicts)
print(f"Accuracy константной модели {base_accuracy:.3f}")

Accuracy константной модели 0.897


In [14]:
count_vect = CountVectorizer(stop_words=stopwords)
n_gramm_train = count_vect.fit_transform(features_train)
n_gramm_test = count_vect.transform(features_test)

print("Размер train'a:", n_gramm_train.shape)
print("Размер test'a:", n_gramm_test.shape)

Размер train'a: (80000, 100804)
Размер test'a: (20000, 100804)


In [15]:
%time

classificator = LogisticRegression(max_iter=200, solver='saga')
grid_params = [{'C':[x for x in range(10,15)]}]
skf_grid=StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
model_lr = GridSearchCV(classificator, grid_params, scoring='f1', cv=skf_grid)
model_lr.fit(features_train_vec, target_train)
LR_best_params = model_lr.best_params_
print(LR_best_params)
print()
means = model_lr.cv_results_['mean_test_score']
stds = model_lr.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model_lr.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
print()

f1_lr = max(means)
print('F1 на обучении', f1_lr)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs
{'C': 10}

0.771763 for {'C': 10}
0.771060 for {'C': 11}
0.771036 for {'C': 12}
0.770342 for {'C': 13}
0.770480 for {'C': 14}

F1 на обучении 0.7717633588058406


In [16]:
%time
predict_lr = model_lr.predict(features_test_vec)
f1_lr_test = f1_score(target_test, predict_lr)
print('F1 на предсказании', f1_lr_test)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs
F1 на предсказании 0.7636264037250069


In [17]:
%time
skf= StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
model=LogisticRegression(max_iter=200, solver='saga')
grid_params = [{'clf__C':[x for x in range(10,15)]}]

pipe = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', model)])

model_lr1=GridSearchCV(pipe, grid_params, scoring='f1', cv=skf)

model_lr1.fit(features_train, target_train)
LR_best_params = model_lr1.best_params_
print(LR_best_params)
print()
means = model_lr1.cv_results_['mean_test_score']
stds = model_lr1.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model_lr1.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
print()

f1_lr1 = max(means)
print('F1 на обучении', f1_lr1)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
{'clf__C': 12}

0.777778 for {'clf__C': 10}
0.777805 for {'clf__C': 11}
0.778088 for {'clf__C': 12}
0.777904 for {'clf__C': 13}
0.777734 for {'clf__C': 14}

F1 на обучении 0.7780876878767267


In [21]:
%time
predict_lr1 = model_lr1.predict(features_test)
f1_lr1_test = f1_score(target_test, predict_lr1)
print('F1 на предсказании', f1_lr1_test)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
F1 на предсказании 0.7662730019225488


Проверим модель на адекватность. Рассчитаем метрику accuracy и сравним её с константной моделью

In [22]:
accuracy_lr = accuracy_score(target_test, predict_lr)
print(f"Accuracy на логистической регрессии {accuracy_lr:.3f}, больше, чем на константной модели")

Accuracy на логистической регрессии 0.957, больше, чем на константной модели


In [23]:
%%time
classificator = DecisionTreeClassifier()
grid_params = [{'max_depth':[x for x in range(20,30)], 'random_state':[12345]}]
model_dtc = GridSearchCV(classificator, grid_params, scoring='f1',cv=skf_grid)
model_dtc.fit(features_train_vec, target_train)
DTC_best_params = model_dtc.best_params_
print(DTC_best_params)
print()
means = model_dtc.cv_results_['mean_test_score']
stds = model_dtc.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model_dtc.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
print()

f1_dtc = max(means)
print('F1 на обучении', f1_dtc)

{'max_depth': 29, 'random_state': 12345}

0.663547 for {'max_depth': 20, 'random_state': 12345}
0.664676 for {'max_depth': 21, 'random_state': 12345}
0.666723 for {'max_depth': 22, 'random_state': 12345}
0.671790 for {'max_depth': 23, 'random_state': 12345}
0.675034 for {'max_depth': 24, 'random_state': 12345}
0.678460 for {'max_depth': 25, 'random_state': 12345}
0.680089 for {'max_depth': 26, 'random_state': 12345}
0.682630 for {'max_depth': 27, 'random_state': 12345}
0.682648 for {'max_depth': 28, 'random_state': 12345}
0.687829 for {'max_depth': 29, 'random_state': 12345}

F1 на обучении 0.6878293382142265
CPU times: user 4min 41s, sys: 527 ms, total: 4min 42s
Wall time: 4min 42s


In [24]:
predict_dtc = model_dtc.predict(features_test_vec)
f1_dtc_test = f1_score(target_test, predict_dtc)
print('F1 на предсказании', f1_dtc_test)

F1 на предсказании 0.6793893129770991


In [25]:
%time
model = DecisionTreeClassifier()
grid_params = [{'clf__max_depth':[x for x in range(20,30)], 'clf__random_state':[12345]}]

pipe = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', model)])

model_dtc1 = GridSearchCV(pipe, grid_params, scoring='f1',cv=skf_grid)
model_dtc1.fit(features_train, target_train)
DTC_best_params = model_dtc1.best_params_
print(DTC_best_params)
print()
means = model_dtc1.cv_results_['mean_test_score']
stds = model_dtc1.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model_dtc1.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
print()

f1_dtc1 = max(means)
print('F1 на обучении', f1_dtc1)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
{'clf__max_depth': 29, 'clf__random_state': 12345}

0.660755 for {'clf__max_depth': 20, 'clf__random_state': 12345}
0.663235 for {'clf__max_depth': 21, 'clf__random_state': 12345}
0.664525 for {'clf__max_depth': 22, 'clf__random_state': 12345}
0.668176 for {'clf__max_depth': 23, 'clf__random_state': 12345}
0.667775 for {'clf__max_depth': 24, 'clf__random_state': 12345}
0.672113 for {'clf__max_depth': 25, 'clf__random_state': 12345}
0.674520 for {'clf__max_depth': 26, 'clf__random_state': 12345}
0.678105 for {'clf__max_depth': 27, 'clf__random_state': 12345}
0.678874 for {'clf__max_depth': 28, 'clf__random_state': 12345}
0.682157 for {'clf__max_depth': 29, 'clf__random_state': 12345}

F1 на обучении 0.6821570409656083


In [26]:
predict_dtc1 = model_dtc1.predict(features_test)
f1_dtc_test1 = f1_score(target_test, predict_dtc1)
print('F1 на предсказании', f1_dtc_test1)

F1 на предсказании 0.667262436699434


In [27]:
#CatBoostClassifier

%time

model_cbc = CatBoostClassifier(verbose=False, iterations=500)
model_cbc.fit(features_train_vec, target_train)
f1_cbc = cross_val_score(model_cbc, features_train_vec, target_train, cv=skf_grid, scoring='f1').mean()
print('F1 на обучении', f1_cbc)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
F1 на обучении 0.7450174795590504


In [28]:
predict_cbc = model_cbc.predict(features_test_vec)
f1_cbc_test = f1_score(target_test, predict_cbc)
print('F1 на предсказании', f1_cbc_test)

F1 на предсказании 0.7297532656023221


In [29]:
#SGDClassifier
%time

classificator = SGDClassifier(max_iter=1000)
grid_params = [{'loss':['hinge', 'modified_huber'], 'learning_rate':['constant', 'optimal', 'adaptive'],
                'eta0':[0.05, 0.1, 0.2, 0.3, 0.5], 'random_state':[12345]}]

model_sgdc = GridSearchCV(classificator, grid_params, scoring='f1',cv=skf_grid)
model_sgdc.fit(features_train_vec, target_train)
SGDC_best_params = model_sgdc.best_params_
print(SGDC_best_params)
print()
print("Grid scores on development set:")
print()
means = model_sgdc.cv_results_['mean_test_score']
stds = model_sgdc.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model_sgdc.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
print()

f1_sgdc = max(means)
print('F1 на обучении', f1_sgdc)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
{'eta0': 0.1, 'learning_rate': 'constant', 'loss': 'modified_huber', 'random_state': 12345}

Grid scores on development set:

0.662168 for {'eta0': 0.05, 'learning_rate': 'constant', 'loss': 'hinge', 'random_state': 12345}
0.743316 for {'eta0': 0.05, 'learning_rate': 'constant', 'loss': 'modified_huber', 'random_state': 12345}
0.664597 for {'eta0': 0.05, 'learning_rate': 'optimal', 'loss': 'hinge', 'random_state': 12345}
0.743976 for {'eta0': 0.05, 'learning_rate': 'optimal', 'loss': 'modified_huber', 'random_state': 12345}
0.665405 for {'eta0': 0.05, 'learning_rate': 'adaptive', 'loss': 'hinge', 'random_state': 12345}
0.744053 for {'eta0': 0.05, 'learning_rate': 'adaptive', 'loss': 'modified_huber', 'random_state': 12345}
0.657762 for {'eta0': 0.1, 'learning_rate': 'constant', 'loss': 'hinge', 'random_state': 12345}
0.744958 for {'eta0': 0.1, 'learning_rate': 'constant', 'loss': 'modified_huber', 'random_state': 12345}
0.

In [30]:
predict_sgdc = model_sgdc.predict(features_test_vec)
f1_sgdc_test = f1_score(target_test, predict_sgdc)
print('F1 на предсказании', f1_sgdc_test)

F1 на предсказании 0.7355251672970615


In [ ]:
#SGDClassifier
#%time

#model = SGDClassifier(max_iter=1000)
#grid_params = [{'clf__loss':['hinge', 'modified_huber'], 'clf__learning_rate':['constant', 'optimal', 'adaptive'],
                #'clf__eta0':[0.05, 0.1, 0.2, 0.3, 0.5], 'clf__random_state':[12345]}]
#pipe = Pipeline([('vect', CountVectorizer()),
                    #('tfidf', TfidfTransformer()),
                    #('clf', model)])

#model_sgdc1 = GridSearchCV(pipe, grid_params, scoring='f1',cv=skf_grid)
#model_sgdc1.fit(features_train, target_train)
#SGDC_best_params = model_sgdc1.best_params_
#print(SGDC_best_params)
#print()
#print("Grid scores on development set:")
#print()
#means = model_sgdc1.cv_results_['mean_test_score']
#stds = model_sgdc1.cv_results_['std_test_score']
#for mean, std, params in zip(means, stds, model_sgdc1.cv_results_['params']):
    #print("%0.6f for %r"% (mean, params))
#print()

#f1_sgdc1 = max(means)
#print('F1 на обучении', f1_sgdc1)

In [ ]:
#predict_sgdc1 = model_sgdc1.predict(features_test)
#f1_sgdc_test1 = f1_score(target_test, predict_sgdc1)
#print('F1 на предсказании', f1_sgdc_test1)

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Добавила паплайн для этой модели, показатель ухудшился на выборке 25000, поэтому запинила код, а в итоговую таблицу вставила в эту графу 'ниже чем без паплайна'
</div>

## Выводы

In [31]:
table=pd.DataFrame({'название модели': [LogisticRegression(), DecisionTreeClassifier(), CatBoostClassifier(), SGDClassifier()], 
                     'f1 для модели при обучении': [f1_lr, f1_dtc, f1_cbc, f1_sgdc],
                     'f1 для модели при обучении с паплайном': [f1_lr1, f1_dtc1, f1_cbc, 'ниже чем без паплайна'],
                     'f1 при предсказании': [f1_lr_test, f1_dtc_test, f1_cbc_test, f1_sgdc_test],
                     'f1 при предсказании с паплайном': [f1_lr1_test, f1_dtc_test1, f1_cbc_test, 'ниже чем без паплайна']
                     })
display(table)

,название модели,f1 для модели при обучении,f1 для модели при обучении с паплайном,f1 при предсказании,f1 при предсказании с паплайном
0,LogisticRegression(),0.771763,0.778088,0.763626,0.766273
1,DecisionTreeClassifier(),0.687829,0.682157,0.679389,0.667262
2,<catboost.core.CatBoostClassifier object at 0x...,0.745017,0.745017,0.729753,0.729753
3,SGDClassifier(),0.744958,ниже чем без паплайна,0.735525,ниже чем без паплайна


Поскольку из-за объема данных у меня умирало ядро, то я сделала проект на части данных. 

Была произведена токенизация, лемматизация текста в датасете. Также были убраны стопслова и произведена векторизация.
После векторизации были обучены 4 модели c кроссвалидацией и подбором гиперпараметров: LogisticRegression(), DecisionTreeClassifier(), CatBoostClassifier(), SGDClassifier()
Наилучшие результаты по параметру F1 у LogisticRegression() и SGDClassifier(), а у  SGDClassifier() при предсказании по параметру F1 показала наилучший результат (0.771134 для сэмпла данных 15000). А для сэмпла данных из 50000, лучший результат у LogisticRegression() - 0.7606. Для сэмпла данных из 100000, лучший результат также у LogisticRegression() 0.763626 и при обучении с паплайном результат чуть выше: 0.766273